In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('train_noshow.csv')

In [7]:
data.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No_show
0,2.987250e+13,5642903,F,2016-04-29 18:38:08,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0
1,5.589978e+14,5642503,M,2016-04-29 16:08:27,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0
2,4.262962e+12,5642549,F,2016-04-29 16:19:04,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,0
3,8.679512e+11,5642828,F,2016-04-29 17:29:31,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0
4,8.841186e+12,5642494,F,2016-04-29 16:07:23,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0


In [8]:
imp_features = ['Gender','Age','Neighbourhood','Scholarship','Hipertension','Diabetes','Alcoholism','Handcap','SMS_received']

In [10]:
imp_data = data[imp_features]

In [11]:
from sklearn.preprocessing import LabelEncoder

In [13]:
for col in ['Gender','Neighbourhood']:
    le = LabelEncoder()
    imp_data[col] = le.fit_transform(imp_data[col])

C:\Users\ZekeLabs\Anaconda3-N\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
rg = RandomForestClassifier(n_estimators=10)

In [17]:
rg.fit(imp_data,data.No_show)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:
from sklearn.model_selection import train_test_split

In [21]:
trainX, testX, trainY, testY = train_test_split(imp_data,data.No_show)

In [22]:
rg.fit(trainX,trainY)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [23]:
rg.score(testX,testY)

0.70900915368556283

In [27]:
len(imp_data.columns)

9

In [51]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.initializers import Initializer
from keras.layers import Dense,Dropout
from keras.utils import to_categorical
from keras.callbacks import Callback
from keras import losses

In [59]:
model = Sequential()
model.add(Dense(40,input_dim = 9, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(20,activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(10,activation = 'relu'))
model.add(Dropout(0.4))
#model.add(Dense(25,activation='relu'))
model.add(Dense(2, activation = 'sigmoid'))
#model.summary()

model.compile(loss='binary_crossentropy',optimizer = 'sgd', metrics = ['accuracy'])

In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 40)                400       
_________________________________________________________________
dropout_6 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 20)                820       
_________________________________________________________________
dropout_7 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 10)                210       
_________________________________________________________________
dropout_8 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 22        
Total para

In [61]:
class history_loss(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        batch_loss = logs.get('loss')
        self.losses.append(batch_loss)

In [62]:
n_epochs = 25
batch_size = 1000
validation_split = 0.2

history_relu = history_loss()

Y = to_categorical(data.No_show)

model.fit(imp_data.as_matrix() , Y , epochs=n_epochs,batch_size=batch_size, callbacks=[history_relu],
          verbose = 2)

Epoch 1/25
 - 7s - loss: 1.1518 - acc: 0.6358
Epoch 2/25
 - 2s - loss: 0.6533 - acc: 0.7052
Epoch 3/25
 - 1s - loss: 0.6236 - acc: 0.7319
Epoch 4/25
 - 1s - loss: 0.6068 - acc: 0.7450
Epoch 5/25
 - 1s - loss: 0.5946 - acc: 0.7529
Epoch 6/25
 - 2s - loss: 0.5882 - acc: 0.7570
Epoch 7/25
 - 2s - loss: 0.5822 - acc: 0.7604
Epoch 8/25
 - 2s - loss: 0.5772 - acc: 0.7622
Epoch 9/25
 - 2s - loss: 0.5740 - acc: 0.7631
Epoch 10/25
 - 2s - loss: 0.5712 - acc: 0.7637
Epoch 11/25
 - 2s - loss: 0.5692 - acc: 0.7642
Epoch 12/25
 - 2s - loss: 0.5659 - acc: 0.7647
Epoch 13/25
 - 2s - loss: 0.5633 - acc: 0.7646
Epoch 14/25
 - 2s - loss: 0.5608 - acc: 0.7646
Epoch 15/25
 - 2s - loss: 0.5605 - acc: 0.7649
Epoch 16/25
 - 2s - loss: 0.5581 - acc: 0.7650
Epoch 17/25
 - 2s - loss: 0.5566 - acc: 0.7651
Epoch 18/25
 - 2s - loss: 0.5555 - acc: 0.7651
Epoch 19/25
 - 2s - loss: 0.5548 - acc: 0.7650
Epoch 20/25
 - 2s - loss: 0.5540 - acc: 0.7651
Epoch 21/25
 - 2s - loss: 0.5534 - acc: 0.7651
Epoch 22/25
 - 2s - lo